In [ ]:
import sys
sys.path.append('..')
import mesh, periodic_homogenization, tensors, mode_viewer, numpy as np
from tri_mesh_viewer import TriMeshViewer as Viewer

### Load and analyze a 2D microstructure using periodic homogenization

In [ ]:
m = mesh.Mesh('../../examples/meshes/2D_microstructure.msh', degree=2)

In [ ]:
view = Viewer(m)
view.showWireframe()
view.avoidRedrawFlicker = True
view.show()

In [ ]:
# Run periodic homogenization for a base material with E=200, nu=0.35
Cbase = tensors.ElasticityTensor2D(200, 0.35)
homogenizationResult = periodic_homogenization.homogenize(m, Cbase, orthotropicCell=True)
print(homogenizationResult.Ch)

### Visualize deformations/stresses of the elastic metamaterial.

In [ ]:
# Compute deformation and strain of homogenized material's minimum energy eigenmode
hr = homogenizationResult
u, e = periodic_homogenization.probe(m, hr, tensors.SymmetricMatrix(hr.Ch.computeEigenstrains().eigenstrains[0]))
stress = Cbase.doubleContract(e)

In [ ]:
view.update(scalarField=stress.vonMises())

In [ ]:
view.update(scalarField=[np.max(s[0]) for s in stress.eigendecomposition()]) # maximum principal stress

In [ ]:
view.update(scalarField=[np.max(np.abs(s[0])) for s in stress.eigendecomposition()]) # maximum abs principal stress

In [ ]:
view.update(vectorField=u) # maximum abs principal stress

In [ ]:
# Visualize the lowest energy eigenmode with an animation.
mview = mode_viewer.ModeViewer(m, u.flatten(), [hr.Ch.computeEigenstrains().eigenvalues[0]], amplitude=0.1)
mview.show()

### Load and analyze a 3D microstructure using periodic homogenization.
The microstructure we analyze here has reflectional symmetries, and for efficiency, we analyze only the "orthotropic base cell" (i.e., the positive octant) using MeshFEM's "ortho base cell homogenization" features. At the end of this notebook we demonstrate that this analysis produces the same result as running standard periodic homogenization on the full cell.

In [ ]:
m3d = mesh.Mesh('../../examples/meshes/3D_microstructure_orthocell.msh', degree=2)

In [ ]:
view3d = Viewer(m3d)
view3d.showWireframe(True)
view3d.avoidRedrawFlicker = True
view3d.show()

In [ ]:
Cbase = tensors.ElasticityTensor3D(200, 0.35)
periodic_homogenization.benchmark_reset()
hr3d = periodic_homogenization.homogenize(m3d, Cbase, orthotropicCell=True)
periodic_homogenization.benchmark_report()
print(hr3d.Ch)

In [ ]:
# Compute deformation and strain of homogenized material's minimum energy eigenmode
ed = hr3d.Ch.computeEigenstrains()
eigenstrains, eigenvalues = ed.eigenstrains, ed.eigenvalues
u3d, e3d = periodic_homogenization.probe(m3d, hr3d, tensors.SymmetricMatrix(eigenstrains[0]))

In [ ]:
# Visualize von Mises stress under the minimum energy eigenmode.
view3d.wireframeMaterial().color = '#57B'
view3d.update(scalarField=Cbase.doubleContract(e3d).vonMises())

In [ ]:
# Use a different colormap for visualizing stress.
import vis, matplotlib
sfield = vis.fields.ScalarField(m3d, Cbase.doubleContract(e3d).vonMises(), colormap=matplotlib.cm.viridis)
view3d.showWireframe(False)
view3d.update(scalarField=sfield)

In [ ]:
# Compute metamaterial's deformation for all its energy eigemodes.
modes = np.column_stack([periodic_homogenization.probe(m3d, hr3d, tensors.SymmetricMatrix(es))[0].ravel() for es in eigenstrains])

In [ ]:
# Visualize one of these modal displacements as a vector field.
view3d.update(vectorField=modes[:,3].reshape((-1, 3)))

### Visualize all the vibrational modes of the 3D elastic metamaterial.

In [ ]:
mview3d = mode_viewer.ModeViewer(m3d, modes, eigenvalues, amplitude=0.1)
mview3d.show()

### Compare against periodic homogenization on the full unit cell.
We demonstrate that the more efficient homogenization routine operating on the positive octant of a structure with reflectional symmetries produces the same result as running traditional periodic homogenization on the full unit cell.

In [ ]:
m3d_full = mesh.Mesh('../../examples/meshes/3D_microstructure.msh', degree=2)

In [ ]:
periodic_homogenization.benchmark_reset()
hr3d_full = periodic_homogenization.homogenize(m3d_full, Cbase)
periodic_homogenization.benchmark_report()

In [ ]:
print(f'Traditional periodic homogenization result:\n{hr3d_full.Ch}')
print(f'Orthotropic base cell homogenization result:\n{hr3d.Ch}')
print(f'Moduli discrepancy: {np.linalg.norm(np.array(hr3d_full.Ch.getOrthotropicParameters()) - np.array(hr3d.Ch.getOrthotropicParameters())) / np.linalg.norm(hr3d.Ch.getOrthotropicParameters())}')